# Training EfficientNet B3 for Skin Cancer Detection

This notebook contains the steps for training and fine-tuning the EfficientNet B3 model on the preprocessed skin lesion images.

## 1. Import Libraries

First, we need to import the necessary libraries.

In [1]:
import os
import numpy as np
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B0 as EFNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import mixed_precision
import json

C:\Users\rikul\.conda\envs\py310\lib\site-packages\tensorflow\python\keras\engine\training_arrays_v1.py:37: UserWarning: A NumPy version >=1.23.5 and <2.5.0 is required for this version of SciPy (detected version 1.21.6)
  from scipy.sparse import issparse  # pylint: disable=g-import-not-at-top


In [2]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

tf.config.run_functions_eagerly(True)

1 Physical GPUs, 1 Logical GPU


## 2. Define Directories

Next, we define the directories for the processed data and the saved models.

In [3]:
BASE_DIR = r"G:\OneDrive\ML-MinorProject\ISIC-2020 Dataset"
PROCESSED_TRAIN_DIR = os.path.join(BASE_DIR, "processed_train")
PROCESSED_VAL_DIR = os.path.join(BASE_DIR, "processed_val")
PROCESSED_TEST_DIR = os.path.join(BASE_DIR, "processed_test")

MODEL_DIR = r"G:\OneDrive\ML-MinorProject\models"
EFFICIENTNET_MODEL_PATH = os.path.join(MODEL_DIR, "efficientnet", "efficientnet_best_model.h5")

## 3. Image Processing Configurations

We define the image processing configurations and create data generators.

In [4]:
IMG_SIZE = (224, 224)  # EfficientNetB0 input size
BATCH_SIZE = 16

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    PROCESSED_TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    PROCESSED_VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    PROCESSED_TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 26500 images belonging to 2 classes.
Found 6626 images belonging to 2 classes.
Found 10982 images belonging to 2 classes.


## 4. Visualize Data Augmentation

We visualize some augmented images to ensure that the augmentations are applied correctly.

In [5]:
# Visualize some augmented images
def visualize_augmentation(generator):
    batch = next(generator)
    images = batch[0]
    
    plt.figure(figsize=(15, 5))
    for i in range(7):
        plt.subplot(1, 7, i + 1)
        plt.imshow(images[i])
        plt.axis('off')
    plt.suptitle("Augmented Images")
    plt.show()

visualize_augmentation(train_generator)

NameError: name 'plt' is not defined

## 5. Enable Mixed Precision

We enable mixed precision to improve training performance.

In [6]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  NVIDIA GeForce GTX 1050 Ti, compute capability 6.1
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


## 6. Define and Compile Model

We define and compile the EfficientNet B3 model architecture.

In [7]:
base_model = EFNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(1, activation='sigmoid', dtype='float32')  # Ensure output is float32
])

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-b0 (Function  (None, 7, 7, 1280)       5919312   
 al)                                                             
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 5,920,593
Trainable params: 5,859,985
Non-trainable params: 60,608
_________________________________________________________________


## 7. Define Callbacks

We define callbacks to improve the training process.

In [8]:
checkpoint = ModelCheckpoint(
    EFFICIENTNET_MODEL_PATH,
    monitor="val_loss",
    save_best_only=True,
    mode="min"
)

early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=10,
    mode="min",
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    patience=5,
    min_lr=1e-6,
    mode="min",
    verbose=1
)

callbacks = [checkpoint, early_stopping, reduce_lr]

## 8. Train the Model

We train the model using the data generators.

In [10]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=callbacks
)


Epoch 1/20
1657/1657 [==============================] - 2581s 2s/step - loss: 0.0559 - accuracy: 0.9833 - val_loss: 0.0746 - val_accuracy: 0.9826 - lr: 2.0000e-05
Epoch 2/20
1657/1657 [==============================] - 2590s 2s/step - loss: 0.0548 - accuracy: 0.9833 - val_loss: 0.0743 - val_accuracy: 0.9819 - lr: 2.0000e-05
Epoch 3/20
1657/1657 [==============================] - 2612s 2s/step - loss: 0.0538 - accuracy: 0.9832 - val_loss: 0.0748 - val_accuracy: 0.9814 - lr: 2.0000e-05
Epoch 4/20
1657/1657 [==============================] - 2523s 2s/step - loss: 0.0525 - accuracy: 0.9838 - val_loss: 0.0745 - val_accuracy: 0.9808 - lr: 2.0000e-05
Epoch 5/20
1657/1657 [==============================] - 2694s 2s/step - loss: 0.0495 - accuracy: 0.9846 - val_loss: 0.0771 - val_accuracy: 0.9784 - lr: 2.0000e-05
Epoch 6/20
1657/1657 [==============================] - 2538s 2s/step - loss: 0.0503 - accuracy: 0.9843 - val_loss: 0.0746 - val_accuracy: 0.9804 - lr: 2.0000e-05
Epoch 7/20
1657/1657 [

## 9. Fine-Tune the Model

We unfreeze the base model and fine-tune the entire model.

In [11]:
# Unfreeze the base model
base_model.trainable = True

# Recompile the model with a lower learning rate
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
fine_tune_history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=callbacks
)

Epoch 1/10
1657/1657 [==============================] - 2746s 2s/step - loss: 0.0536 - accuracy: 0.9836 - val_loss: 0.0774 - val_accuracy: 0.9813 - lr: 1.0000e-05
Epoch 2/10
1657/1657 [==============================] - 2520s 2s/step - loss: 0.0519 - accuracy: 0.9841 - val_loss: 0.0755 - val_accuracy: 0.9804 - lr: 1.0000e-05
Epoch 3/10
1657/1657 [==============================] - 2688s 2s/step - loss: 0.0514 - accuracy: 0.9838 - val_loss: 0.0781 - val_accuracy: 0.9798 - lr: 1.0000e-05
Epoch 4/10
1657/1657 [==============================] - 2635s 2s/step - loss: 0.0503 - accuracy: 0.9838 - val_loss: 0.0773 - val_accuracy: 0.9799 - lr: 1.0000e-05
Epoch 5/10
1657/1657 [==============================] - 2760s 2s/step - loss: 0.0497 - accuracy: 0.9845 - val_loss: 0.0767 - val_accuracy: 0.9786 - lr: 1.0000e-05
Epoch 6/10
1657/1657 [==============================] - 2708s 2s/step - loss: 0.0479 - accuracy: 0.9847 - val_loss: 0.0776 - val_accuracy: 0.9817 - lr: 1.0000e-05
Epoch 7/10
1657/1657 [

## 10. Save the Final Model

We save the final model after training and fine-tuning.

In [12]:
final_model_path = os.path.join(MODEL_DIR, "efficientnet", "efficientnet_final_model.h5")
model.save(final_model_path)
print(f"Final model saved to {final_model_path}")

Final model saved to G:\OneDrive\ML-MinorProject\models\efficientnet\efficientnet_final_model.h5
